To-do list:
- In the Overview, add some useful tables (e.g. number of companies per industry, number of companies by size, number of companies by HQ, etc...)
- Check the other projects I have done and borrow some functions.


Change year from float to datetime

### Appendix: code validation

Use the code below to track how some rows change as you apply changes to the whole dataset.

In [ ]:
# edit this to make the companies regexes
validation_filter = {
    "comp_name": [
        "Avance Gas Holding ltd",
        "Prosafe SE",
        "Seadrill Ltd.",
        "Tallink",
        "ICA Gruppen AB",
    ]
}
validation_cols_to_show = None  # 'None' shows all columns in df by default

<center><span style="font-size:30px; font-weight: bold;">Nordic Compass Database</span></center>
<center><span style="font-size:24px;">Analysis of ESG Performance and CSRD Compliance</span></center>

<center><span style="font-size:22px;"><b>Section 1:</b> Preprocessing and cleaning </span></center>

## Introduction to this section

Insert discussion here...

## Imports

In [2]:
import pandas as pd
import numpy as np
from rapidfuzz import fuzz, process
import sys
import os

sys.path.append(os.path.abspath(".."))
import random
from functions import (
    display_unique_counts,
    show_missing_values,
    test_filter,
    test_ticker,
    test_company,
    find_similar_entries,
)

In [67]:
# load the file
df = pd.read_csv("../datasets/NordicCompass2014_2022.csv")

## Overview of dataset

The database is too large to focus on all columns necessary for full compliance with CSRD. I will focus only on columns relating to a company's environmental performance.

In [4]:
# df.columns.tolist()

relevant_columns = [
    "comp_name",
    "ticker",
    "year",
    "segment",
    "industry",
    "hq_country",
    "ceo_sust_statem",
    "sales",
    "env_policy",
    "ep_targets",
    "env_impact_red",
    "energy_consump",
    "incr_renew_en",
    "disclosure_raw",
    "resource_target",
    "water_withdraw",
    "water_disclose",
    "ghg_emis",
    "transport_emis",
    "audit_es_report",
    "su_guidelines",
    "su_aud_disclose",
    "su_eva_disclose",
    "su_env_assess",
]

In [69]:
# selects all rows and only relevant columns
df = df.loc[:, relevant_columns]

In [70]:
df.head()

,comp_name,ticker,year,segment,industry,hq_country,ceo_sust_statem,sales,env_policy,ep_targets,...,resource_target,water_withdraw,water_disclose,ghg_emis,transport_emis,audit_es_report,su_guidelines,su_aud_disclose,su_eva_disclose,su_env_assess
0,Archer Ltd.,ARCHER,2016.0,Mid,Oil & Gas,Bermuda,N,841.9,N,Y,...,N,ND,N,ND,ND,N,N,N,N,N
1,Archer Ltd.,ARCHER,2017.0,Mid,Oil & Gas,Bermuda,Y,705.7,Y,N,...,N,ND,N,ND,ND,N,N,N,N,N
2,Archer Ltd.,ARCHO,2020.0,Mid,Energy,Norway,Y,735.7142857,Y,Y,...,Y,ND,N,ND,ND,Y,Y,Y,N,N
3,AutoStore Holdings Ltd.,AUTO,2021.0,Large,Industrials,Bermuda,Y,292.5,Y,N,...,N,ND,N,0.7366,371.9243,N,Y,N,Y,N
4,Avance Gas Holding ltd,AVACF,2019.0,Mid,Energy,Norway,Y,223.5901786,Y,Y,...,N,ND,N,N,N,N,N,N,N,N


In [8]:
display_unique_counts(df)

Unique companies in the database:  782
Unique tickers in the database:  783


## Cleaning the data

Process:
1) Check that companies and tickers match. Some companies have multiple tickers per company, some tickers have multiple companies per ticker.
2) Check for duplicates (e.g. same company, same year appearing multiple times)
3) Where data is duplicated, prioritise data where 'GHG emissions' data exists and/or 'sales' are higher (some have net income reported by mistake)

### 1) Standardise company names and tickers

a) For each ticker, associate it with only one company 

# Consider turning this into a function

In [9]:
# Show rows where a single ticker is associated with multiple companies (e.g. 'BDRILL' = 'Borr Drilling Ltd' and 'Borr Drilling Ltd.')
companies_per_ticker_df = (
    df.groupby("ticker")["comp_name"].nunique().sort_values(ascending=False)
)

tickers_with_multiple_companies_index = companies_per_ticker_df[
    companies_per_ticker_df > 1
].index

tickers_multi_comp_df = df[df["ticker"].isin(tickers_with_multiple_companies_index)]

tickers_multi_comp_list = []

for ticker, other_cols in tickers_multi_comp_df.groupby("ticker"):
    companies = ", ".join(sorted(other_cols["comp_name"].unique()))
    tickers_multi_comp_list.append(f"{ticker}: {companies}")

tickers_multi_comp_list = "\n".join(tickers_multi_comp_list)

print(
    "Tickers associated with multiple companies: ",
    len(tickers_with_multiple_companies_index),
    end="\n\n",
)

print(tickers_multi_comp_list)

Tickers associated with multiple companies:  114

ACR: Axactor, Axactor SE
ADE: Adevinta, Adevinta ASA
AF: ÅF AB, ÅF Pöyry AB
AKRBP: Aker BP ASA, Aker BP ASA (Det norske oljeselskap ASA)
AKTIA: Aktia Bank PLC (formerly known as Aktia, Aktia Bank Plc (formerly known as Aktia Pankki Oyj), Aktia Bank plc
AM1: Ahlstrom-Munksjö Oyj, Ahlstrom-Munksjö Oyj  (Munksjö Oyj)
ANOD: AddNode Group AB, Addnode Group AB
ARCUS: Arcus ASA, Arcus asa
ARION: Arion Banki hf., Arion Banki hf. SDB
ASSA: ASSA ABLOY AB, Assa Abloy AB
AZTO: ArcticZymes Technologies (Biotec Pharmacon ), ArcticZymes Technologies ASA(formerly Biotec Pharmacon ASA)
BALD: Balder Fastighets AB, Fastighets AB Balder
BDRILL: Borr Drilling Ltd, Borr Drilling Ltd.
BHG: BHG (formerly Bygghemma Group First AB), BHG AB(formerly Bygghemma Group First AB), BHG Group AB (Bygghemma Group First AB)
BITTI: Bittium Oyj, Bittium Oyj  (Elektrobit Oyj)
BO: Bang & Olufsen A/S, Bang & Olufsen Holding A/S
BWLPG: BW LPG, BW LPG Ltd
BWO: BW Offshore Limite

In [10]:
test_ticker(df, "BDRILL")

,comp_name,ticker,year,segment,industry,hq_country,ceo_sust_statem,sales,env_policy,ep_targets,...,resource_target,water_withdraw,water_disclose,ghg_emis,transport_emis,audit_es_report,su_guidelines,su_aud_disclose,su_eva_disclose,su_env_assess
7,Borr Drilling Ltd,BDRILL,2019.0,Small,Energy,Norway,NaN,291.8485523,Y,N,...,Y,ND,N,150.784,43.671,N,Y,N,N,N
8,Borr Drilling Ltd,BDRILL,2021.0,Mid,Oil & Gas,Bermuda,NaN,219.2857143,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Borr Drilling Ltd,BDRILL,2022.0,Mid,Oil & Gas,Bermuda,NaN,291.5345028,Y,N,...,N,ND,N,ND,ND,N,Y,N,N,Y
10,Borr Drilling Ltd,BDRILL,2020.0,Large,Oil & Gas,Bermuda,Y,274.5535714,Y,Y,...,Y,ND,N,163.841,ND,N,Y,Y,Y,Y
11,Borr Drilling Ltd.,BDRILL,2017.0,Large,Oil & Gas,Norway,N,0.1,Y,N,...,N,ND,N,ND,ND,N,N,N,N,N
12,Borr Drilling Ltd.,BDRILL,2018.0,Large,Oil & Gas,Norway,N,143.3913043,Y,N,...,N,ND,Y,185472000,ND,N,Y,N,N,N


In [11]:
# find the most recent row for each ticker. Store the company name from that row
# e.g. if 'ACR' is 'Axactor' in 2020 and 'Axactor SE' in 2021, then 'Axactor SE' will be stored

ticker_latest_company_df = df.sort_values(
    by=["ticker", "year"], ascending=[True, False]
).drop_duplicates(subset=["ticker"], keep="first")

company_mapping_dict = dict(
    zip(ticker_latest_company_df["ticker"], ticker_latest_company_df["comp_name"])
)

# access the ticker column and apply the map operation to each value in that column
# (in this case, 'comp_name' values will be replaced with the values from the ticker_latest_company_df, through the company_mapping_dict dictionary)
df.loc[:, "comp_name"] = df["ticker"].map(company_mapping_dict)

Validate that all tickers have only one associated company.

In [12]:
companies_per_ticker_df = (
    df.groupby("ticker")["comp_name"].nunique().sort_values(ascending=False)
)
tickers_with_multiple_companies_index = companies_per_ticker_df[
    companies_per_ticker_df > 1
].index

print(
    "Tickers associated with multiple companies: ",
    len(tickers_with_multiple_companies_index),
    end="\n\n",
)
display_unique_counts(df)

Tickers associated with multiple companies:  0

Unique companies in the database:  666
Unique tickers in the database:  783


In [13]:
# proof that 'BDRILL' is now associated with only one company
test_ticker(df, "BDRILL")["comp_name"].unique()

array(['Borr Drilling Ltd'], dtype=object)

b) For each company, associate it with only one ticker. 

In [14]:
# show rows where a single company is associated with multiple tickers (e.g. 'Archer Ltd.' = 'ARCHER' and 'ARCHO')
tickers_per_company_df = (
    df.groupby("comp_name")["ticker"].nunique().sort_values(ascending=False)
)
companies_with_multiple_tickers_index = tickers_per_company_df[
    tickers_per_company_df > 1
].index

print(
    "Companies associated with multiple tickers: ",
    len(companies_with_multiple_tickers_index),
    end="\n\n",
)

companies_multi_ticker_df = df[
    df["comp_name"].isin(companies_with_multiple_tickers_index)
]

companies_multi_ticker_list = []

for company, other_cols in companies_multi_ticker_df.groupby("comp_name"):
    tickers = ", ".join(sorted(other_cols["ticker"].unique()))
    companies_multi_ticker_list.append(f"{company}: {tickers}")

companies_multi_ticker_list = "\n".join(companies_multi_ticker_list)

print(companies_multi_ticker_list)

Companies associated with multiple tickers:  111

A.P. Møller -Maersk A/S: MAERSK, MAERSK A
ABG Sundal Collier Holding ASA: ABG, ASC
Akastor  ASA: AKAST, AKKVF
Aker BP ASA: AKERBP, AKRBP
Ambu A/S: AMBU, AMBU-B
Archer Ltd.: ARCHER, ARCHO
Asetek A/S: ASETEK, ASTK
Avance Gas Holding ltd: AGAS, AVACF
Axactor SE: ACR, AXA
BankNordik P/F: BNORDIK, BNORDIK CSE
Beijer Alma AB: BEIA, BEIA B
Beijer Ref AB: BEIJ, BEIJ B
Belships ASA: BEL, BELO
Bonheur ASA: BON, BONH
Borregaard ASA: BRG, BRGO
Bouvet ASA: BOUV, BOUVET
Carlsberg A/S: CARL, CARL B
Caverion Oyj: CAV, CAV1V
CellaVision AB: CEVI, SEVI
Cloetta AB: CLA, CLA B
Coloplast A/S: COLO, COLO B
ContextVision: CONTX, COVO
Corem Property Group AB: CORE, CORE A
Crayon Group Holding ASA: CRAYN, CRAYNO
DOF ASA: DOF, DOFO
Elanders AB: ELAN, ELAN B
Frontline Ltd: FRO, FROo
Genmab A/S: GEN, GMAB
H. Lundbeck A/S: HLUN, LUN
Hexagon AB: HEXA, HEXA B
Huhtamäki Oyj: HUH, HUH1V
Höegh LNG Holdings: HLNG, ND
Investor AB: INVE, INVE B
Kemira Oyj: KEMIRA, KRA
Kesk

In [15]:
test_company(df, "Archer Ltd.")["ticker"].unique()

array(['ARCHER', 'ARCHO'], dtype=object)

In [16]:
# find the most recent row for each company.
# Store the ticker from that row (e.g. if 'Archer Ltd.' is 'ARCHER' in 2017 and 'ARCHO' in 2020, then 'ARCHO' will be stored)
company_latest_ticker_df = df.sort_values(
    by=["comp_name", "year"], ascending=[True, False]
).drop_duplicates(subset=["comp_name"], keep="first")

ticker_mapping = dict(
    zip(company_latest_ticker_df["comp_name"], company_latest_ticker_df["ticker"])
)

df.loc[:, "ticker"] = df["comp_name"].map(ticker_mapping)

In [17]:
test_company(df, "Archer Ltd.")

,comp_name,ticker,year,segment,industry,hq_country,ceo_sust_statem,sales,env_policy,ep_targets,...,resource_target,water_withdraw,water_disclose,ghg_emis,transport_emis,audit_es_report,su_guidelines,su_aud_disclose,su_eva_disclose,su_env_assess
0,Archer Ltd.,ARCHO,2016.0,Mid,Oil & Gas,Bermuda,N,841.9,N,Y,...,N,ND,N,ND,ND,N,N,N,N,N
1,Archer Ltd.,ARCHO,2017.0,Mid,Oil & Gas,Bermuda,Y,705.7,Y,N,...,N,ND,N,ND,ND,N,N,N,N,N
2,Archer Ltd.,ARCHO,2020.0,Mid,Energy,Norway,Y,735.7142857,Y,Y,...,Y,ND,N,ND,ND,Y,Y,Y,N,N


In [18]:
# validate that all tickers have only one associated company
tickers_per_company_df = (
    df.groupby("comp_name")["ticker"].nunique().sort_values(ascending=False)
)
companies_with_multiple_tickers_index = tickers_per_company_df[
    tickers_per_company_df > 1
].index

print(
    "Companies associated with multiple tickers: ",
    len(companies_with_multiple_tickers_index),
)

Companies associated with multiple tickers:  0


In [19]:
test_company(df, "ICA Gruppen AB").head(1)

,comp_name,ticker,year,segment,industry,hq_country,ceo_sust_statem,sales,env_policy,ep_targets,...,resource_target,water_withdraw,water_disclose,ghg_emis,transport_emis,audit_es_report,su_guidelines,su_aud_disclose,su_eva_disclose,su_env_assess
1440,ICA Gruppen AB,NaN,2022.0,NaN,Retail,Sweden,NaN,12818.29236,Y,Y,...,Y,ND,N,16.275,78.318,N,Y,N,N,Y


In [20]:
# ICA was missing a ticker, so I filled it in
df.loc[df["comp_name"] == "ICA Gruppen AB", "ticker"] = "ICA"

In [21]:
test_company(df, "ICA Gruppen AB")

,comp_name,ticker,year,segment,industry,hq_country,ceo_sust_statem,sales,env_policy,ep_targets,...,resource_target,water_withdraw,water_disclose,ghg_emis,transport_emis,audit_es_report,su_guidelines,su_aud_disclose,su_eva_disclose,su_env_assess
1440,ICA Gruppen AB,ICA,2022.0,NaN,Retail,Sweden,NaN,12818.29236,Y,Y,...,Y,ND,N,16.275,78.318,N,Y,N,N,Y
2568,ICA Gruppen AB,ICA,2014.0,Large,Consumer Services,Sweden,Y,9156.9,Y,Y,...,Y,ND,N,237.5,146.803,N,Y,N,Y,Y
2569,ICA Gruppen AB,ICA,2015.0,Large,Consumer Services,Sweden,Y,11074.5,Y,Y,...,Y,ND,N,149.734,143.897,Y,Y,ND,Y,Y
2570,ICA Gruppen AB,ICA,2016.0,Large,Consumer Services,Sweden,Y,10831.2,Y,Y,...,Y,ND,N,128.604,125.851,Y,Y,N,Y,Y
2571,ICA Gruppen AB,ICA,2017.0,Large,Consumer Services,Sweden,Y,10807.6,Y,Y,...,Y,ND,N,85.2,98.3,Y,Y,N,Y,N
2572,ICA Gruppen AB,ICA,2018.0,Large,Consumer Services,Sweden,Y,10934.02844,Y,Y,...,Y,ND,N,74.659,72.607,Y,N,0.0079,Y,Y
2573,ICA Gruppen AB,ICA,2019.0,Large,Grocery Stores,Sweden,Y,11359.2649,Y,Y,...,Y,N,N,52.689,75.842,Y,Y,N,N,Y
2574,ICA Gruppen AB,ICA,2020.0,Large,Consumer Staples,Sweden,Y,12024.66197,Y,Y,...,Y,ND,N,19.394,72.727,N,Y,Y,Y,Y
2575,ICA Gruppen AB,ICA,2021.0,Large,"Personal Care, Drug and Grocery Stores",Sweden,Y,12184.06018,Y,Y,...,Y,ND,ND,17.347,61.444,Y,Y,Y,Y,Y
2576,ICA Gruppen AB,ICA,NaN,0,Industrials,Sweden,NaN,Y,Y,3787000,...,0,266,8110,Y,0,Y,Y,0.000446809,Y,Y


In [22]:
display_unique_counts(df)

Unique companies in the database:  666
Unique tickers in the database:  666


c) Catch other companies that might have slipped through the net

First use the find_similar_entries function to catch pairs above the similarity threshold.

In [23]:
similar_pairs_df = find_similar_entries(df, 75)

print(f"Number of rows: {len(similar_pairs_df)}", end="\n")
similar_pairs_df

Number of rows: 42


,entry1,year1,entry2,year2,similarity
0,"SCA, Svenska Cellulosa AB (SCA)",2022.0,SCA. Svenska Cellulosa AB (SCAA),2021.0,95.238095
1,Hagar hf (HAGA),2022.0,Hagar hf. (HAGAR),2015.0,93.750000
2,Akastor ASA (AKAST),2020.0,Akastor ASA (AKA),2017.0,91.891892
3,"Ericsson, Telefonab. L M (ERIC)",2022.0,Ericsson Telefonab LM (ERIC-B),2019.0,91.803279
4,SpareBank 1 SR-Bank (SRBANK),2022.0,SpareBank 1 SR-Bank ASA (SRBNK),2020.0,91.525424
5,Tanker Investments Ltd (TNK),2020.0,Tanker Investments Ltd. (TIL),2016.0,91.228070
6,Nobia AB (NOBI),2022.0,Nobina AB (NOBINA),2022.0,90.909091
7,"Hennes & Mauritz AB, H & M (HM)",2022.0,Hennes & Mauritz AB. H&M (HM B),2020.0,90.322581
8,Momentum Group (MMGR),2022.0,Momentum Group AB (MMGR B),2020.0,89.361702
9,Avance Gas Holding ltd (AGAS),2020.0,Avance Gas Holding ltd. (AVANCE),2016.0,88.524590


Not all of the companies above are the same company, so I only want to change the ones that are.

In [24]:
# is there a more robust way of doing this? Probably.
# Write comp_names instead...
indices_to_keep = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 14, 15, 16, 19, 20]

similar_pairs_df = similar_pairs_df.loc[indices_to_keep]

similar_pairs_df[["comp_name1", "ticker1"]] = similar_pairs_df["entry1"].str.extract(
    r"^(.*) \((.*)\)$"
)
similar_pairs_df[["comp_name2", "ticker2"]] = similar_pairs_df["entry2"].str.extract(
    r"^(.*) \((.*)\)$"
)

replacement_mapping = {
    "comp_name": dict(
        zip(similar_pairs_df["comp_name2"], similar_pairs_df["comp_name1"])
    ),
    "ticker": dict(zip(similar_pairs_df["ticker2"], similar_pairs_df["ticker1"])),
}

df["comp_name"] = df["comp_name"].replace(replacement_mapping["comp_name"])
df["ticker"] = df["ticker"].replace(replacement_mapping["ticker"])

In [25]:
test_filter(
    df, {"comp_name": ["Catella AB", "Catena AB", "Hagar hf", "Hagar hf."]}, "comp_name"
)["comp_name"].unique()

array(['Hagar hf', 'Catella AB', 'Catena AB'], dtype=object)

In [26]:
# could use an 'override' column with 1s and 0s--create an Excel file for the business person to decide. Then load...

I now want to catch the last companies that are the same, but may have names that weren't caught by the similarity checker. I will do this manually.

In [27]:
# sorted(df['comp_name'].unique().tolist())

In [28]:
# To get the final companies that appear multiple times under different names, I checked manually and compiled a list
similar_companies_manual = [
    "Kindred Group Plc (formerly Unibet Group)",
    "Kindred Group plc",
    "Ahlstrom Oyj",
    "Ahlstrom-Munksjö Oyj",
    "Bergman & Beving AB",
    "Bergman & Beving AB  (B&B Tools AB)",
    "Kinnevik AB",
    "Kinnevik AB  (Kinnevik Investment AB)",
    "MT Højgaard A/S (formerly known as Højga",
    "MT Højgaard Holding A/S  (Højgaard Holding A/S)",
    "Metso Outotec Oyj",
    "Metso Outotec Oyj  (Outotec Oyj)",
    "Nordnet AB",
    "Nordnet AB publ",
    "Radisson Hospitality AB",
    "Radisson Hospitality AB  (Rezidor Hotel Group AB)",
    "Revenio Group Corporation",
    "Revenio Group Oyj",
    "Raisio Oyj",
    "Raisio Oyj Vaihto-osake",
    "Royal Caribbean Cruises Ltd.",
    "Royal Caribbean Group (formerly: Royal Caribbean Cruises Ltd)",
    "TORM A/S",
    "TORM plc",
    "VBG GROUP AB",
    "VBG Group AB",
]

filtered_df = df[df["comp_name"].isin(similar_companies_manual)]

latest_entries = filtered_df.sort_values(by="year", ascending=False).drop_duplicates(
    subset=["comp_name"], keep="first"
)

# Merge back to get ticker and ensure entry1 has the most recent year
similar_pairs_manual = []
for company1, company2 in zip(
    similar_companies_manual[::2], similar_companies_manual[1::2]
):
    entry1 = latest_entries[latest_entries["comp_name"] == company1]
    entry2 = latest_entries[latest_entries["comp_name"] == company2]

    if not entry1.empty and not entry2.empty:
        # Extract relevant details
        year1 = entry1["year"].values[0]
        year2 = entry2["year"].values[0]
        ticker1 = entry1["ticker"].values[0]
        ticker2 = entry2["ticker"].values[0]

        # Ensure entry1 has the most recent year
        if year1 < year2:
            company1, company2 = company2, company1
            year1, year2 = year2, year1
            ticker1, ticker2 = ticker2, ticker1

        # Format the entries
        formatted_entry1 = f"{company1} [{ticker1}]"
        formatted_entry2 = f"{company2} [{ticker2}]"

        # Append to the list
        similar_pairs_manual.append((formatted_entry1, year1, formatted_entry2, year2))

# Convert to DataFrame
similar_pairs_manual_df = pd.DataFrame(
    similar_pairs_manual, columns=["entry1", "year1", "entry2", "year2"]
)

similar_pairs_manual_df

,entry1,year1,entry2,year2
0,Ahlstrom-Munksjö Oyj [AM1],2020.0,Ahlstrom Oyj [AHL1V],2016.0
1,Bergman & Beving AB [BERG],2022.0,Bergman & Beving AB (B&B Tools AB) [BBTO],2015.0
2,MT Højgaard A/S (formerly known as Højga [MTHH],2022.0,MT Højgaard Holding A/S (Højgaard Holding A/S...,2020.0
3,Metso Outotec Oyj [METSO],2022.0,Metso Outotec Oyj (Outotec Oyj) [OTE],2019.0
4,Nordnet AB [SAVE],2022.0,Nordnet AB publ [NN],2020.0
5,Radisson Hospitality AB [RADH],2017.0,Radisson Hospitality AB (Rezidor Hotel Group ...,2016.0
6,Revenio Group Corporation [REG1V],2022.0,Revenio Group Oyj [REG],2020.0
7,Raisio Oyj Vaihto-osake [RAIVV],2022.0,Raisio Oyj [RAI],2020.0
8,TORM plc [TRMD],2022.0,TORM A/S [Torm A],2015.0


In [29]:
# now integrate all remaining similar companies
# note: I have done this twice now. Can I turn this into a function? def map_similar_pairs (just make sure to edit df name, as I changed it...)
# store regex part as a variable (used multiple times)
similar_pairs_manual_df[["comp_name1", "ticker1"]] = similar_pairs_manual_df[
    "entry1"
].str.extract(r"^(.*) \[(.*)\]$")
similar_pairs_manual_df[["comp_name2", "ticker2"]] = similar_pairs_manual_df[
    "entry2"
].str.extract(r"^(.*) \[(.*)\]$")

# Step 2: Create mappings for replacement
comp_name_mapping = dict(
    zip(similar_pairs_manual_df["comp_name2"], similar_pairs_manual_df["comp_name1"])
)
ticker_mapping = dict(
    zip(similar_pairs_manual_df["ticker2"], similar_pairs_manual_df["ticker1"])
)

# Step 3: Replace values in the original DataFrame
df["comp_name"] = df["comp_name"].replace(comp_name_mapping)
df["ticker"] = df["ticker"].replace(ticker_mapping)

In [30]:
# check that the changes were made correctly
# df[df['comp_name'].str.startswith('Ra')].sort_values(by='comp_name', ascending=True)

In [31]:
# df[df.duplicated(subset=['comp_name', 'year'])]
df[df.duplicated(subset=["ticker", "year"], keep=False)].sort_values(
    by=["comp_name", "year"], ascending=[True, False]
)

,comp_name,ticker,year,segment,industry,hq_country,ceo_sust_statem,sales,env_policy,ep_targets,...,resource_target,water_withdraw,water_disclose,ghg_emis,transport_emis,audit_es_report,su_guidelines,su_aud_disclose,su_eva_disclose,su_env_assess
973,Ahlstrom-Munksjö Oyj,AM1,2016.0,Mid,Basic Materials,Sweden,Y,1085.9,Y,Y,...,Y,23000,Y,467.2,ND,Y,Y,N,N,Y
1128,Ahlstrom-Munksjö Oyj,AM1,2016.0,Mid,Basic Materials,Sweden,Y,1142.9,Y,Y,...,N,ND,N,ND,ND,N,Y,N,N,N
972,Ahlstrom-Munksjö Oyj,AM1,2015.0,Mid,Basic Materials,Finland,N,1074.7,Y,Y,...,Y,24000,N,502.2,ND,N,Y,N,N,N
1127,Ahlstrom-Munksjö Oyj,AM1,2015.0,Mid,Basic Materials,Sweden,Y,1130.7,Y,N,...,N,40500,N,337,ND,N,Y,N,Y,Y
971,Ahlstrom-Munksjö Oyj,AM1,2014.0,Mid,Basic Materials,Finland,Y,1001.1,Y,Y,...,Y,26,N,333.4,ND,N,Y,N,N,Y
1126,Ahlstrom-Munksjö Oyj,AM1,2014.0,Mid,Basic Materials,Sweden,Y,1137.3,Y,Y,...,Y,41250,Y,352,ND,N,Y,N,N,Y
1396,Arion Banki hf.,ARION,2019.0,Large,Financials,Iceland,N,430.4688077,Y,N,...,N,136.99,N,0.0634,0.0708,N,N,N,N,N
3433,Arion Banki hf.,ARION,2019.0,Large,Financials,Iceland,Y,354.3743079,Y,Y,...,N,136.99,N,0.1342,0.3154,N,Y,N,N,Y
2126,Prosafe SE,PRS,2020.0,Mid,Oil & Gas,Norway,Y,50.625,Y,Y,...,Y,44289,N,0.1924,1.785,N,Y,0,N,Y
2131,Prosafe SE,PRS,2020.0,Mid,Oil & Gas,Norway,NaN,49.6497373,Y,Y,...,Y,44289,N,ND,ND,N,Y,Y,Y,Y


In [32]:
# is this the most robust way of doing this? I don't think so...just make sure the index doesn't change above!
duplicates_to_drop = [1128, 1127, 1126, 1396, 2131, 1247]
df = df.drop(index=duplicates_to_drop)

In [33]:
df[df.duplicated(subset=["ticker", "year"], keep=False)].sort_values(
    by=["comp_name", "year"], ascending=[True, False]
)

,comp_name,ticker,year,segment,industry,hq_country,ceo_sust_statem,sales,env_policy,ep_targets,...,resource_target,water_withdraw,water_disclose,ghg_emis,transport_emis,audit_es_report,su_guidelines,su_aud_disclose,su_eva_disclose,su_env_assess


In [34]:
display_unique_counts(df)

Unique companies in the database:  642
Unique tickers in the database:  642


In [35]:
# test_company(df, 'ICA Gruppen AB')

### 2) Standardise industry and sector

In [36]:
df["segment"].unique().tolist()

['Mid', 'Large', 'Small', nan, 'ND', '0']

In [37]:
companies_missing_segments = (
    df[df["segment"].isin(["ND", "0", np.nan])]["comp_name"].unique().tolist()
)

companies_missing_segments

['Seadrill Ltd',
 'Basware Oyj',
 'Bakkafrost P/F',
 'Onxeo SA',
 'ICA Gruppen AB',
 'Norwegian Finans Holding',
 'Schibsted ASA']

In [38]:
valid_segments = ["Mid", "Large", "Small"]

most_recent_segments = (
    df[df["comp_name"].isin(companies_missing_segments)][
        df["segment"].isin(valid_segments)
    ]
    .dropna(subset=["segment", "year"])
    .sort_values(by="year", ascending=False)
    .groupby("comp_name")
    .first()["segment"]
    .to_dict()
)

df["segment"] = df.apply(
    lambda row: (
        most_recent_segments.get(row["comp_name"], row["segment"])
        if row["segment"] in ["ND", "0", np.nan]
        else row["segment"]
    ),
    axis=1,
)

C:\Users\james\AppData\Local\Temp\ipykernel_5800\2175919304.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df["comp_name"].isin(companies_missing_segments)][


In [39]:
# verification that this worked
# df[df['comp_name']
# .isin([
# 'Seadrill Ltd',
# 'Basware Oyj',
# 'Bakkafrost P/F',
# 'Onxeo SA',
# 'ICA Gruppen AB',
# 'Norwegian Finans Holding',
# 'Schibsted ASA'])]

In [40]:
test_filter(df)

,comp_name,ticker,year,segment,industry,hq_country,ceo_sust_statem,sales,env_policy,ep_targets,...,resource_target,water_withdraw,water_disclose,ghg_emis,transport_emis,audit_es_report,su_guidelines,su_aud_disclose,su_eva_disclose,su_env_assess
0,Archer Ltd.,ARCHO,2016.0,Mid,Oil & Gas,Bermuda,N,841.9,N,Y,...,N,ND,N,ND,ND,N,N,N,N,N
1,Archer Ltd.,ARCHO,2017.0,Mid,Oil & Gas,Bermuda,Y,705.7,Y,N,...,N,ND,N,ND,ND,N,N,N,N,N
2,Archer Ltd.,ARCHO,2020.0,Mid,Energy,Norway,Y,735.7142857,Y,Y,...,Y,ND,N,ND,ND,Y,Y,Y,N,N
3,AutoStore Holdings Ltd.,AUTO,2021.0,Large,Industrials,Bermuda,Y,292.5,Y,N,...,N,ND,N,0.7366,371.9243,N,Y,N,Y,N
4,Avance Gas Holding ltd,AGAS,2019.0,Mid,Energy,Norway,Y,223.5901786,Y,Y,...,N,ND,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3807,Gaming Innovation Group Inc,GIG,2018.0,Mid,Technology,Malta,Y,151.372,Y,N,...,N,ND,N,ND,ND,N,N,N,N,N
3808,Gaming Innovation Group Inc,GIG,2020.0,Mid,Technology,Malta,Y,63.027,Y,Y,...,Y,ND,N,ND,ND,Y,N,N,N,N
3809,Gaming Innovation Group Inc,GIG,2016.0,Mid,Technology,United States,N,54,N,N,...,N,ND,N,ND,ND,N,N,N,N,N
3810,Gaming Innovation Group Inc,GIG,2017.0,Mid,Technology,Malta,N,120.4,N,N,...,N,ND,N,ND,ND,N,N,N,N,N


In [41]:
# use this as a condition to test edge cases
test_company(df, "Seadrill Ltd")

,comp_name,ticker,year,segment,industry,hq_country,ceo_sust_statem,sales,env_policy,ep_targets,...,resource_target,water_withdraw,water_disclose,ghg_emis,transport_emis,audit_es_report,su_guidelines,su_aud_disclose,su_eva_disclose,su_env_assess
66,Seadrill Ltd,SDRL,2014.0,Large,Oil & Gas,Bermuda,N,3621,N,N,...,N,ND,N,ND,ND,N,N,N,N,N
67,Seadrill Ltd,SDRL,2015.0,Mid,Oil & Gas,Bermuda,N,3977.1,Y,N,...,N,ND,N,ND,ND,N,N,N,N,N
68,Seadrill Ltd,SDRL,2016.0,Mid,Oil & Gas,Bermuda,N,3018.1,N,N,...,N,ND,N,ND,ND,N,N,N,N,N
69,Seadrill Ltd,SDRL,2019.0,Mid,Energy,Norway,NaN,1194.642857,Y,N,...,N,ND,N,ND,ND,N,N,N,N,N
70,Seadrill Ltd,SDRL,2017.0,Small,Energy,Norway,NaN,1748.49477,Y,N,...,N,ND,N,ND,ND,N,N,N,N,N
71,Seadrill Ltd,SDRL,2018.0,Small,Energy,Norway,NaN,1095.289294,Y,N,...,N,ND,N,ND,ND,N,N,N,N,N
72,Seadrill Ltd,SDRL,2020.0,Large,Oil & Gas,United Kingdom,N,945.5357143,Y,N,...,N,ND,N,ND,ND,N,N,N,N,N
73,Seadrill Ltd,SDRL,2022.0,Small,Oil & Gas,Bermuda,NaN,800.5328529,N,N,...,N,ND,N,ND,ND,N,Y,N,N,N


Now extract data from the most recent year for each company and standardise that for all years.

In [42]:
# Step 1: Define invalid values
invalid_values = ["ND"]


# Step 2: Define a function to get the most recent valid value or fallback
def get_most_recent_valid(group, column):
    # Filter out rows where the column is None or contains invalid values
    valid_data = group[~(group[column].isin(invalid_values) | group[column].isna())]
    if not valid_data.empty:
        return valid_data.iloc[0][column]  # Most recent valid value
    else:
        return "Unknown"  # Default fallback


# Step 3: Apply logic for multiple columns
most_recent_data = {}
for comp_name, group in df.sort_values(by="year", ascending=False).groupby("comp_name"):
    most_recent_data[comp_name] = {
        "segment": get_most_recent_valid(group, "segment"),
        "industry": get_most_recent_valid(group, "industry"),
        "hq_country": get_most_recent_valid(group, "hq_country"),
    }

# Step 4: Map the most recent valid data back to the original DataFrame
df["segment"] = df["comp_name"].map(lambda x: most_recent_data[x]["segment"])
df["industry"] = df["comp_name"].map(lambda x: most_recent_data[x]["industry"])
df["hq_country"] = df["comp_name"].map(lambda x: most_recent_data[x]["hq_country"])

In [43]:
# verify that the above code did its job
test_company(df, "Seadrill Ltd")

,comp_name,ticker,year,segment,industry,hq_country,ceo_sust_statem,sales,env_policy,ep_targets,...,resource_target,water_withdraw,water_disclose,ghg_emis,transport_emis,audit_es_report,su_guidelines,su_aud_disclose,su_eva_disclose,su_env_assess
66,Seadrill Ltd,SDRL,2014.0,Small,Oil & Gas,Bermuda,N,3621,N,N,...,N,ND,N,ND,ND,N,N,N,N,N
67,Seadrill Ltd,SDRL,2015.0,Small,Oil & Gas,Bermuda,N,3977.1,Y,N,...,N,ND,N,ND,ND,N,N,N,N,N
68,Seadrill Ltd,SDRL,2016.0,Small,Oil & Gas,Bermuda,N,3018.1,N,N,...,N,ND,N,ND,ND,N,N,N,N,N
69,Seadrill Ltd,SDRL,2019.0,Small,Oil & Gas,Bermuda,NaN,1194.642857,Y,N,...,N,ND,N,ND,ND,N,N,N,N,N
70,Seadrill Ltd,SDRL,2017.0,Small,Oil & Gas,Bermuda,NaN,1748.49477,Y,N,...,N,ND,N,ND,ND,N,N,N,N,N
71,Seadrill Ltd,SDRL,2018.0,Small,Oil & Gas,Bermuda,NaN,1095.289294,Y,N,...,N,ND,N,ND,ND,N,N,N,N,N
72,Seadrill Ltd,SDRL,2020.0,Small,Oil & Gas,Bermuda,N,945.5357143,Y,N,...,N,ND,N,ND,ND,N,N,N,N,N
73,Seadrill Ltd,SDRL,2022.0,Small,Oil & Gas,Bermuda,NaN,800.5328529,N,N,...,N,ND,N,ND,ND,N,Y,N,N,N


Merge similar industries under one name

In [44]:
industry_mapping = {
    "Oil & Gas": "Energy",
    "Oil & Gas Equipment & Services": "Energy",
    "Industrials": "Industrial Goods and Services",
    "Personal Care, Drug and Grocery Stores": "Consumer Goods and Services",
    "Consumer Goods": "Consumer Goods and Services",
    "Consumer Discretionary": "Consumer Goods and Services",
    "Consumer Services": "Consumer Goods and Services",
    "Consumer Staples": "Consumer Goods and Services",
    "Basic Resources": "Basic Materials",
    "Financial Services": "Finance",
    "Financials": "Finance",
    "Healthcare": "Health Care",
}

# Apply the mapping to the 'industry' column
df["industry"] = df["industry"].replace(industry_mapping)

sorted(df["industry"].unique().tolist())

['Basic Materials',
 'Biotechnology',
 'Consumer Goods and Services',
 'Energy',
 'Finance',
 'Health Care',
 'Industrial Goods and Services',
 'Leisure',
 'Media',
 'Real Estate',
 'Retail',
 'Technology',
 'Telecommunications',
 'Travel and Leisure',
 'Unknown',
 'Utilities']

In [45]:
df["hq_country"] = df["hq_country"].str.strip().replace({"UK": "United Kingdom"})

sorted(df["hq_country"].unique().tolist())

['America',
 'Belgium',
 'Bermuda',
 'Canada',
 'Cayman Islands',
 'Chile',
 'Cyprus',
 'Denmark',
 'Estonia',
 'Faroe Islands',
 'Finland',
 'France',
 'Germany',
 'Iceland',
 'Jersey',
 'Luxembourg',
 'Malta',
 'Netherlands',
 'Norway',
 'Sweden',
 'Switzerland',
 'United Arab Emirates',
 'United Kingdom',
 'United States',
 'Virgin Islands, British']

### 3) Deal with missing values

In [46]:
df = df.dropna(subset=["year"])

In [47]:
df.shape

(3805, 24)

In [48]:
# Add total values for each column in the data
show_missing_values(df)

,Missing Values,Missing Percentage,'ND' Values,'ND' Percentage
cols,,,,
comp_name,0,0.00,0,0.00
ticker,0,0.00,0,0.00
year,0,0.00,0,0.00
segment,0,0.00,0,0.00
industry,0,0.00,0,0.00
hq_country,0,0.00,0,0.00
ceo_sust_statem,499,13.11,2,0.05
sales,19,0.50,0,0.00
env_policy,23,0.60,2,0.05


In [49]:
# Make 'ND' more robust to any later aggregation
df.replace("ND", np.nan, inplace=True)

# Convert 'Y' and 'N' to dummy variables
df.replace({"Y": 1, "N": 0}, inplace=True)

C:\Users\james\AppData\Local\Temp\ipykernel_5800\1324403236.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace("ND", np.nan, inplace=True)
C:\Users\james\AppData\Local\Temp\ipykernel_5800\1324403236.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace({"Y": 1, "N": 0}, inplace=True)


In [50]:
df.shape

(3805, 24)

In [51]:
show_missing_values(df)

,Missing Values,Missing Percentage,'ND' Values,'ND' Percentage
cols,,,,
comp_name,0,0.00,0,0.0
ticker,0,0.00,0,0.0
year,0,0.00,0,0.0
segment,0,0.00,0,0.0
industry,0,0.00,0,0.0
hq_country,0,0.00,0,0.0
ceo_sust_statem,501,13.17,0,0.0
sales,19,0.50,0,0.0
env_policy,25,0.66,0,0.0


In [52]:
# Create a summary DataFrame
summary_df = pd.DataFrame(
    {
        "Column Name": df.columns,
        "Data Type": [df[col].dtype for col in df.columns],
        "1s": [(df[col] == 1).sum() for col in df.columns],
        "0s": [(df[col] == 0).sum() for col in df.columns],
    }
)

# Reset the index for better readability
summary_df.reset_index(drop=True, inplace=True)

summary_df

,Column Name,Data Type,1s,0s
0,comp_name,object,0,0
1,ticker,object,0,0
2,year,float64,0,0
3,segment,object,0,0
4,industry,object,0,0
5,hq_country,object,0,0
6,ceo_sust_statem,object,2306,995
7,sales,object,0,0
8,env_policy,object,3407,370
9,ep_targets,object,2636,1142


In [53]:
filtered_df = df[
    (df["energy_consump"].isin([1, 0]))
    | (df["water_withdraw"].isin([1, 0]))
    | (df["ghg_emis"].isin([1, 0]))
    | (df["transport_emis"].isin([1, 0]))
]

df[["energy_consump", "water_withdraw", "ghg_emis", "transport_emis"]] = df[
    ["energy_consump", "water_withdraw", "ghg_emis", "transport_emis"]
].replace({1: np.nan, 0: np.nan})

C:\Users\james\AppData\Local\Temp\ipykernel_5800\2543934469.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["energy_consump", "water_withdraw", "ghg_emis", "transport_emis"]] = df[


# Change years_es_reporting to a count of values per company per year

In [54]:
# Create a summary DataFrame
summary_df = pd.DataFrame(
    {
        "Column Name": df.columns,
        "Data Type": [df[col].dtype for col in df.columns],
        "1s": [(df[col] == 1).sum() for col in df.columns],
        "0s": [(df[col] == 0).sum() for col in df.columns],
    }
)

# Reset the index for better readability
summary_df.reset_index(drop=True, inplace=True)

summary_df

,Column Name,Data Type,1s,0s
0,comp_name,object,0,0
1,ticker,object,0,0
2,year,float64,0,0
3,segment,object,0,0
4,industry,object,0,0
5,hq_country,object,0,0
6,ceo_sust_statem,object,2306,995
7,sales,object,0,0
8,env_policy,object,3407,370
9,ep_targets,object,2636,1142


In [55]:
df.loc[df["comp_name"] == "Truecaller AB", "ticker"] = "TRUE B"
# df[df['comp_name']=='Truecaller AB']

In [56]:
df = df.drop(df[df["year"] < 2019].index)

df["year"].value_counts()

year
2020.0    491
2019.0    486
2021.0    439
2022.0    422
Name: count, dtype: int64

# CLEAN su_aud_disclose

In [57]:
columns_to_clean = [
    "ceo_sust_statem",
    "env_policy",
    "env_impact_red",
    "water_disclose",
    "audit_es_report",
    "su_guidelines",
    "su_env_assess",
]

for col in columns_to_clean:
    # Convert 'T', 't', 'Y', 'y' to 1
    df[col] = df[col].apply(lambda x: 1 if str(x).upper() in ["T", "Y"] else x)

    # Convert blanks to NaN
    df[col] = df[col].replace(r"^\s*$", np.nan, regex=True)

    # Convert any value that's not 0 or 1 to NaN
    df[col] = df[col].apply(lambda x: x if pd.isna(x) or x in [0, 1] else np.nan)

In [58]:
columns_to_fix = ["sales", "transport_emis"]

for col in columns_to_fix:
    # Convert values to numeric (non-numeric values become NaN)
    df[col] = pd.to_numeric(df[col], errors="coerce")

    # Replace values less than or equal to 0 with NaN
    df[col] = df[col].apply(lambda x: np.nan if x <= 0 else x)

## New columns

In [59]:
df.columns

Index(['comp_name', 'ticker', 'year', 'segment', 'industry', 'hq_country',
       'ceo_sust_statem', 'sales', 'env_policy', 'ep_targets',
       'env_impact_red', 'energy_consump', 'incr_renew_en', 'disclosure_raw',
       'resource_target', 'water_withdraw', 'water_disclose', 'ghg_emis',
       'transport_emis', 'audit_es_report', 'su_guidelines', 'su_aud_disclose',
       'su_eva_disclose', 'su_env_assess'],
      dtype='object')

In [60]:
# base_year

earliest_year = df.groupby("comp_name")["year"].min()

earliest_year

df = df.join(earliest_year, on="comp_name", how="left", rsuffix="_base")
df["base_year"] = df["year_base"]
df = df.drop("year_base", axis=1)

df.head()

,comp_name,ticker,year,segment,industry,hq_country,ceo_sust_statem,sales,env_policy,ep_targets,...,water_withdraw,water_disclose,ghg_emis,transport_emis,audit_es_report,su_guidelines,su_aud_disclose,su_eva_disclose,su_env_assess,base_year
2,Archer Ltd.,ARCHO,2020.0,Mid,Energy,Norway,1.0,735.714286,1.0,1,...,NaN,0.0,NaN,NaN,1.0,1.0,1,0,0.0,2020.0
3,AutoStore Holdings Ltd.,AUTO,2021.0,Large,Industrial Goods and Services,Bermuda,1.0,292.500000,1.0,0,...,NaN,0.0,0.7366,371.9243,0.0,1.0,0,1,0.0,2021.0
4,Avance Gas Holding ltd,AGAS,2019.0,Mid,Energy,Norway,1.0,223.590179,1.0,1,...,NaN,0.0,NaN,NaN,0.0,0.0,0,0,0.0,2019.0
5,Avance Gas Holding ltd,AGAS,2020.0,Mid,Energy,Norway,1.0,183.675000,1.0,1,...,NaN,0.0,NaN,NaN,1.0,1.0,0,0,0.0,2019.0
7,Borr Drilling Ltd,BDRILL,2019.0,Mid,Energy,Bermuda,NaN,291.848552,1.0,0,...,NaN,0.0,150.784,43.6710,0.0,1.0,0,0,0.0,2019.0


In [61]:
# # fix this

# # 1. Create 'years_esg_data' by counting the number of rows for each 'comp_name'
# df['years_esg_data'] = df.groupby('comp_name')['year'].transform('count')

# # 2. Create 'consecutive_years_esg_data' by checking consecutive years starting from 2022
# def calculate_consecutive_years(group):
#     # Create a set of years for the current 'comp_name'
#     years = set(group['year'])

#     # Start from 2022 and count consecutive years backwards
#     count = 0
#     for year in range(2022, 2019, -1):  # Checking years 2022, 2021, 2020, ...
#         if year in years:
#             count += 1
#         else:
#             break  # Stop if any year is missing in the consecutive sequence

#     return count

# # Apply the function to each group of 'comp_name'
# df['consecutive_years_esg_data'] = df.groupby('comp_name').apply(calculate_consecutive_years).reset_index(level=0, drop=True)

# # Optionally, display the results
# df[['comp_name', 'year', 'years_esg_data', 'consecutive_years_esg_data']].head()

In [62]:
df = df.rename(columns={"sales": "revenue_EUR"})

df.head()

,comp_name,ticker,year,segment,industry,hq_country,ceo_sust_statem,revenue_EUR,env_policy,ep_targets,...,water_withdraw,water_disclose,ghg_emis,transport_emis,audit_es_report,su_guidelines,su_aud_disclose,su_eva_disclose,su_env_assess,base_year
2,Archer Ltd.,ARCHO,2020.0,Mid,Energy,Norway,1.0,735.714286,1.0,1,...,NaN,0.0,NaN,NaN,1.0,1.0,1,0,0.0,2020.0
3,AutoStore Holdings Ltd.,AUTO,2021.0,Large,Industrial Goods and Services,Bermuda,1.0,292.500000,1.0,0,...,NaN,0.0,0.7366,371.9243,0.0,1.0,0,1,0.0,2021.0
4,Avance Gas Holding ltd,AGAS,2019.0,Mid,Energy,Norway,1.0,223.590179,1.0,1,...,NaN,0.0,NaN,NaN,0.0,0.0,0,0,0.0,2019.0
5,Avance Gas Holding ltd,AGAS,2020.0,Mid,Energy,Norway,1.0,183.675000,1.0,1,...,NaN,0.0,NaN,NaN,1.0,1.0,0,0,0.0,2019.0
7,Borr Drilling Ltd,BDRILL,2019.0,Mid,Energy,Bermuda,NaN,291.848552,1.0,0,...,NaN,0.0,150.784,43.6710,0.0,1.0,0,0,0.0,2019.0


In [63]:
# First convert both columns to numeric
df["ghg_emis"] = pd.to_numeric(df["ghg_emis"], errors="coerce")
df["revenue_EUR"] = pd.to_numeric(df["revenue_EUR"], errors="coerce")

# Then perform the division
df["ghg_emis_per_EUR_revenue"] = df["ghg_emis"] / df["revenue_EUR"]

df.head(1)

,comp_name,ticker,year,segment,industry,hq_country,ceo_sust_statem,revenue_EUR,env_policy,ep_targets,...,water_disclose,ghg_emis,transport_emis,audit_es_report,su_guidelines,su_aud_disclose,su_eva_disclose,su_env_assess,base_year,ghg_emis_per_EUR_revenue
2,Archer Ltd.,ARCHO,2020.0,Mid,Energy,Norway,1.0,735.714286,1.0,1,...,0.0,NaN,NaN,1.0,1.0,1,0,0.0,2020.0,NaN


In [64]:
# First convert both columns to numeric
df["water_withdraw"] = pd.to_numeric(df["water_withdraw"], errors="coerce")
df["revenue_EUR"] = pd.to_numeric(df["revenue_EUR"], errors="coerce")

# Then perform the division
df["water_withdraw_per_EUR_revenue"] = df["water_withdraw"] / df["revenue_EUR"]

In [65]:
# folder_path = r"C:\Users\james\OneDrive - University of Aberdeen\01 - Turing College\\D99 - Capstone Project\ESG Ratings Project - Nordic Compass"

# df.to_csv(f'{folder_path}/nordic_compass_df_modified.csv', index=False)

In [66]:
df.columns

Index(['comp_name', 'ticker', 'year', 'segment', 'industry', 'hq_country',
       'ceo_sust_statem', 'revenue_EUR', 'env_policy', 'ep_targets',
       'env_impact_red', 'energy_consump', 'incr_renew_en', 'disclosure_raw',
       'resource_target', 'water_withdraw', 'water_disclose', 'ghg_emis',
       'transport_emis', 'audit_es_report', 'su_guidelines', 'su_aud_disclose',
       'su_eva_disclose', 'su_env_assess', 'base_year',
       'ghg_emis_per_EUR_revenue', 'water_withdraw_per_EUR_revenue'],
      dtype='object')

# To do:

#### Cleaning
Remove whitespace/commas from all values

Converting all 1s and 0s to NaNs for certain columns can all be done at once

Check 'energy_consumption', 'resource_target', 'water_withdraw', 'ghg_emis'  because there are a few nonsensical values



#### New columns


Create 'years_esg_data' and 'consecutive_years_esg_data' columns

Create 'gap analysis: total missing metrics (coverage of metrics)'

Create a column: 'GHG per EUR revenue_ranking_all_companies' - This is binned from 1 to 10 (using quartiles and calculated using only values 
from the same year)

Create a column: 'GHG per EUR revenue_ranking_sector' - This is also binned from 1 to 10 (and calculated using only values from the same year)

Calculate the average GHG per EUR revenue as well as IQR--apply the outlier transformation and put all outliers in the '0' bin

Create a column: 'GHG per EUR revenue_ranking_all_PY' -- This is to compare to the values from the previous year

Create a column: 'GHG per EUR revenue_ranking_sector_PY' -- This is to compare to the values from the previous year

Create a column: '% change in GHG per EUR revenue vs PY'

Create a column: '% change in GHG emissions vs PY'

Create a column: 'transport emissions as a % of total emissions'

Create a column: '% change in transport emissions vs PY'

Create a column: 'Transport emissions as % of total emissions' (compare to sector)


Use the bins only for GHG emissions/EUR--compare values in each bin for all columns...

See how bin values vary from year to year

Calculate the number of companies that have migrated from bin to bin





#### Bonus columns

'GHG intensity reduction % vs sector-specific targets'--normalise so make it a % above or below target

'GHG intensity reduction % vs others in the sector_CY'--also normalise (and consider whether positive is good or bad)



#### Summary columns

Summarise results by:

- Segment/Industry

- HQ country

Declarations per year

--check which industry has the highest % of missing values

Percentage of companies in each industry that have their sustainability work audited